In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import random_split, DataLoader
from torchvision.datasets import ImageFolder
from torchvision import models
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from tqdm import tqdm  # <-- import tqdm for progress bars
#DATA SETUP & TRANSFORMS
dataset_root = r"C:\Users\yozev\OneDrive\Desktop\artFiltered"

transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

full_dataset = ImageFolder(root=dataset_root, transform=transform)
print("Classes:", full_dataset.classes)


# SPLIT DATA INTO TRAIN/VAL/TEST
dataset_size = len(full_dataset)
train_size = int(0.75 * dataset_size)
val_size = int(0.10 * dataset_size)
test_size = dataset_size - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(
    full_dataset, [train_size, val_size, test_size], generator=torch.Generator().manual_seed(42))

print(f"Dataset sizes -> Train: {len(train_dataset)}, Val: {len(val_dataset)}, Test: {len(test_dataset)}")

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


# DEFINE PRE-TRAINED MODEL (ResNet50)
from torchvision.models import ResNet50_Weights

num_classes = len(full_dataset.classes)
model = models.resnet50(weights=ResNet50_Weights.DEFAULT)

# Replace the final fully connected layer to match the number of classes
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Dropout(0.5),  # Dropout added to prevent overfitting
    nn.Linear(num_ftrs, num_classes)
)

# TRAINING SETUP
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

epochs = 25
for epoch in range(epochs):
    print(f"\nEpoch [{epoch+1}/{epochs}]")

    #TRAINING
    model.train()
    running_loss = 0.0


    train_loop = tqdm(train_loader, desc='Training', leave=False)
    for images, labels in train_loop:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()


        train_loop.set_postfix({'loss': f'{loss.item():.4f}'})

    avg_train_loss = running_loss / len(train_loader)

    #VALIDATION
    model.eval()
    val_loss = 0.0


    val_loop = tqdm(val_loader, desc='Validation', leave=False)
    with torch.no_grad():
        for images, labels in val_loop:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            val_loop.set_postfix({'val_loss': f'{loss.item():.4f}'})

    avg_val_loss = val_loss / len(val_loader)
    scheduler.step()

    print(f"Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")


# EVALUATION ON TEST SET
model.eval()
all_preds = []
all_labels = []

test_loop = tqdm(test_loader, desc='Testing')
with torch.no_grad():
    for images, labels in test_loop:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, dim=1)
        all_preds.append(predicted.cpu().numpy())
        all_labels.append(labels.cpu().numpy())

all_preds = np.concatenate(all_preds)
all_labels = np.concatenate(all_labels)

acc = accuracy_score(all_labels, all_preds)
prec = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
rec = recall_score(all_labels, all_preds, average='weighted', zero_division=0)
f1 = f1_score(all_labels, all_preds, average='weighted')

print("\n=== TEST METRICS ===")
print(f"Accuracy:   {acc:.4f}")
print(f"Precision:  {prec:.4f}")
print(f"Recall:     {rec:.4f}")
print(f"F1 Score:   {f1:.4f}")


Classes: ['Abstract_Expressionism', 'Art_Nouveau_Modern', 'Baroque', 'Cubism', 'Expressionism', 'Impressionism', 'Naive_Art_Primitivism', 'Northern_Renaissance', 'Post_Impressionism', 'Realism', 'Rococo', 'Romanticism', 'Symbolism']
Dataset sizes -> Train: 48563, Val: 6475, Test: 9713

Epoch [1/25]


Train Loss: 1.4098 | Val Loss: 1.0338

Epoch [2/25]


Train Loss: 0.9880 | Val Loss: 0.9467

Epoch [3/25]


Train Loss: 0.8246 | Val Loss: 0.9009

Epoch [4/25]


Train Loss: 0.7089 | Val Loss: 0.9068

Epoch [5/25]


...

Train Loss: 0.2271 | Val Loss: 0.9562

Epoch [24/25]


Train Loss: 0.2268 | Val Loss: 0.9637

Epoch [25/25]


Train Loss: 0.2260 | Val Loss: 0.9585


Testing: 100%|██████████| 304/304 [00:32<00:00,  9.44it/s]


=== TEST METRICS ===
Accuracy:   0.7317
Precision:  0.7295
Recall:     0.7317
F1 Score:   0.7298
